# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [41]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform
import numpy as np

In [2]:
data = pd.read_csv('./data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
data_purchased = data[['CustomerID','ProductName','Quantity']]
data_purchased = data_purchased.groupby(['CustomerID','ProductName']).sum()
data_purchased

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1
...                                            ...
98200      Vol Au Vents                         50
           Wasabi Powder                        25
           Wine - Fume Blanc Fetzer             25
           Wine - Hardys Bankside Shiraz        25
           Yogurt - French Vanilla              25

[63628 rows x 1 columns]

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [5]:
data_purchased_pt = pd.pivot_table(data_purchased, values='Quantity', index=['ProductName'],columns=['CustomerID']).fillna(0)
data_purchased_pt.info()

<class 'pandas.core.frame.DataFrame'>
Index: 452 entries, Anchovy Paste - 56 G Tube to Zucchini - Yellow
Columns: 1000 entries, 33 to 98200
dtypes: float64(1000)
memory usage: 3.5+ MB


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [6]:
t_purchased = data_purchased_pt.T

In [7]:
dist_calculation = pdist(X = t_purchased , metric='euclidean')
dist_calculation

array([ 11.91637529,  10.48808848,  11.22497216, ..., 304.13812651,
       305.16389039, 303.10889132])

In [8]:
dist_distribution = squareform(dist_calculation)
dist_distribution

array([[  0.        ,  11.91637529,  10.48808848, ..., 228.62851966,
        239.        , 229.77380181],
       [ 11.91637529,   0.        ,  11.74734012, ..., 228.01096465,
        239.03765394, 229.70415756],
       [ 10.48808848,  11.74734012,   0.        , ..., 228.08112592,
        238.26665734, 229.77380181],
       ...,
       [228.62851966, 228.01096465, 228.08112592, ...,   0.        ,
        304.13812651, 305.16389039],
       [239.        , 239.03765394, 238.26665734, ..., 304.13812651,
          0.        , 303.10889132],
       [229.77380181, 229.70415756, 229.77380181, ..., 305.16389039,
        303.10889132,   0.        ]])

In [9]:
euclid_dist = pd.DataFrame(dist_distribution,
                           index=data_purchased_pt.columns, 
                           columns=data_purchased_pt.columns)

euclid_dist

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.000000,11.916375,10.488088,11.224972,11.401754,11.090537,12.409674,11.045361,11.269428,11.489125,...,206.871941,213.180675,225.656819,198.232187,230.913404,220.501701,217.188858,228.628520,239.000000,229.773802
200,11.916375,0.000000,11.747340,12.083046,12.569805,12.288206,12.165525,12.083046,11.874342,12.000000,...,206.310446,212.635839,224.697575,197.139544,230.952376,220.202180,215.728997,228.010965,239.037654,229.704158
264,10.488088,11.747340,0.000000,11.489125,11.224972,11.445523,12.000000,11.401754,11.180340,11.747340,...,206.387984,212.946003,225.435135,197.600607,230.371439,219.136943,216.612557,228.081126,238.266657,229.773802
356,11.224972,12.083046,11.489125,0.000000,12.083046,11.789826,12.328828,11.135529,11.958261,12.165525,...,206.649462,213.082144,225.452878,197.494304,231.038958,219.952268,217.437347,228.098663,238.493186,229.464594
412,11.401754,12.569805,11.224972,12.083046,0.000000,11.704700,12.328828,11.135529,11.789826,11.747340,...,206.900942,211.679002,225.572605,197.630969,230.614397,219.733930,217.446545,227.997807,238.396728,228.927936
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,220.501701,220.202180,219.136943,219.952268,219.733930,219.599636,219.538152,219.924987,219.827205,220.070443,...,283.945417,283.945417,302.076149,272.717803,278.388218,0.000000,273.861279,291.547595,306.186218,307.205143
98069,217.188858,215.728997,216.612557,217.437347,217.446545,217.425849,216.903204,217.294731,217.080630,216.751009,...,283.945417,283.945417,295.803989,283.945417,285.043856,273.861279,0.000000,287.228132,297.909382,294.745653
98159,228.628520,228.010965,228.081126,228.098663,227.997807,228.197283,228.028507,228.181945,227.868383,228.103047,...,283.945417,279.508497,300.000000,290.473751,300.000000,291.547595,287.228132,0.000000,304.138127,305.163890


In [15]:
euclid_dist_norm = pd.DataFrame(1/(1 + squareform(pdist(t_purchased, 'euclidean'))),
                                index=t_purchased.index,
                                columns=t_purchased.index)

euclid_dist_norm

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.004515,0.004521,0.004543,0.004526,0.004530,0.004533,0.004534,0.004526,0.004528,0.004523,...,0.003509,0.003509,0.003300,0.003653,0.003579,1.000000,0.003638,0.003418,0.003255,0.003245
98069,0.004583,0.004614,0.004595,0.004578,0.004578,0.004578,0.004589,0.004581,0.004585,0.004592,...,0.003509,0.003509,0.003369,0.003509,0.003496,0.003638,1.000000,0.003469,0.003345,0.003381
98159,0.004355,0.004367,0.004365,0.004365,0.004367,0.004363,0.004366,0.004363,0.004369,0.004365,...,0.003509,0.003565,0.003322,0.003431,0.003322,0.003418,0.003469,1.000000,0.003277,0.003266


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [18]:
euclid_dist_norm_2 = euclid_dist_norm.iloc[:,0]
type(euclid_dist_norm_2)

pandas.core.series.Series

In [20]:
top_cust = euclid_dist_norm_2.to_frame()
top_cust

,33
CustomerID,
33,1.000000
200,0.077421
264,0.087047
356,0.081800
412,0.080634
...,...
97928,0.004515
98069,0.004583
98159,0.004355


In [28]:
top_cust = top_cust.sort_values(33, ascending = False)
top_cust

,33
CustomerID,
33,1.000000
3317,0.087047
264,0.087047
3535,0.087047
2503,0.085983
...,...
91777,0.004140
92492,0.004121
95017,0.004077


In [31]:
top_5cust = top_cust.iloc[1:6]

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [32]:
top_5cust

,33
CustomerID,
3317,0.087047
264,0.087047
3535,0.087047
2503,0.085983
3305,0.085638


In [33]:
top_5cust.reset_index(inplace=True)
top_5cust

,CustomerID,33
0,3317,0.087047
1,264,0.087047
2,3535,0.087047
3,2503,0.085983
4,3305,0.085638


In [35]:
top_5cust = top_5cust.iloc[:,0]
top_5cust

0    3317
1     264
2    3535
3    2503
4    3305
Name: CustomerID, dtype: int64

In [36]:
top_5cust = list(top_5cust)
top_5cust

[3317, 264, 3535, 2503, 3305]

In [39]:
data_purchased

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1
...                                            ...
98200      Vol Au Vents                         50
           Wasabi Powder                        25
           Wine - Fume Blanc Fetzer             25
           Wine - Hardys Bankside Shiraz        25
           Yogurt - French Vanilla              25

[63628 rows x 1 columns]

In [65]:
step6 = data_purchased[np.in1d(data_purchased.index.get_level_values(0), top_5cust)]
type(step6)

pandas.core.frame.DataFrame

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [67]:
step6 = step6.reset_index()

In [69]:
step6 = step6[['ProductName', 'Quantity']]

In [72]:
step6_sum = step6.groupby('ProductName').sum().sort_values('Quantity', ascending = False)
step6_sum

,Quantity
ProductName,
Butter - Unsalted,3
Wine - Ej Gallo Sierra Valley,3
Towels - Paper / Kraft,3
Soup - Campbells Bean Medley,3
Wine - Blue Nun Qualitatswein,3
...,...
Hinge W Undercut,1
Ice Cream Bar - Hageen Daz To,1
Jagermeister,1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [73]:
step7 = data_purchased.reset_index()
step7

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1
...,...,...,...
63623,98200,Vol Au Vents,50
63624,98200,Wasabi Powder,25
63625,98200,Wine - Fume Blanc Fetzer,25
63626,98200,Wine - Hardys Bankside Shiraz,25


In [74]:
step7 = step7[step7['CustomerID'] == 33]
step7

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1
5,33,"Beef - Chuck, Boneless",1
6,33,Beef - Prime Rib Aaa,1
7,33,Beer - Original Organic Lager,1
8,33,Beer - Rickards Red,1
9,33,Black Currants,1


In [82]:
step7_result = step6_sum.merge(step7, on='ProductName', how='left')
step7_result

,ProductName,Quantity_x,CustomerID,Quantity_y
0,Butter - Unsalted,3,NaN,NaN
1,Wine - Ej Gallo Sierra Valley,3,NaN,NaN
2,Towels - Paper / Kraft,3,33.0,1.0
3,Soup - Campbells Bean Medley,3,NaN,NaN
4,Wine - Blue Nun Qualitatswein,3,NaN,NaN
...,...,...,...,...
213,Hinge W Undercut,1,NaN,NaN
214,Ice Cream Bar - Hageen Daz To,1,NaN,NaN
215,Jagermeister,1,NaN,NaN
216,Jolt Cola - Electric Blue,1,NaN,NaN


In [83]:
step7_result = step7_result[step7_result['CustomerID']!= 33.0]
step7_result

,ProductName,Quantity_x,CustomerID,Quantity_y
0,Butter - Unsalted,3,NaN,NaN
1,Wine - Ej Gallo Sierra Valley,3,NaN,NaN
3,Soup - Campbells Bean Medley,3,NaN,NaN
4,Wine - Blue Nun Qualitatswein,3,NaN,NaN
6,Chicken - Soup Base,2,NaN,NaN
...,...,...,...,...
211,Halibut - Steaks,1,NaN,NaN
213,Hinge W Undercut,1,NaN,NaN
214,Ice Cream Bar - Hageen Daz To,1,NaN,NaN
215,Jagermeister,1,NaN,NaN


In [84]:
products = step7_result.iloc[:5]
products

,ProductName,Quantity_x,CustomerID,Quantity_y
0,Butter - Unsalted,3,NaN,NaN
1,Wine - Ej Gallo Sierra Valley,3,NaN,NaN
3,Soup - Campbells Bean Medley,3,NaN,NaN
4,Wine - Blue Nun Qualitatswein,3,NaN,NaN
6,Chicken - Soup Base,2,NaN,NaN


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [92]:
products_recommended = {}

for i in range(5):
    euclid_dist_norm_2 = euclid_dist_norm.iloc[:,i]
    top_cust = euclid_dist_norm_2.to_frame()
    column_name = top_cust.columns
    top_cust = top_cust.sort_values(column_name[0], ascending = False)
    top_5cust = top_cust.iloc[0:6]
    top_5cust.reset_index(inplace=True)
    top_5cust = top_5cust[top_5cust['CustomerID']!= column_name[0]]
    top_5cust = top_5cust.iloc[:,0]
    top_5cust = list(top_5cust)
    step6 = data_purchased[np.in1d(data_purchased.index.get_level_values(0), top_5cust)]
    step6 = step6.reset_index()
    step6 = step6[['ProductName', 'Quantity']]
    step6_sum = step6.groupby('ProductName').sum().sort_values('Quantity', ascending = False)
    step7 = data_purchased.reset_index()
    step7 = step7[step7['CustomerID'] == column_name[0]]
    step7_result = step6_sum.merge(step7, on='ProductName', how='left')
    step7_result = step7_result[step7_result['CustomerID']!= column_name[0]]
    products = step7_result.iloc[:5]
    products_recommended[column_name[0]] = list(products['ProductName'])
    

print(products_recommended)

{33: ['Butter - Unsalted', 'Wine - Ej Gallo Sierra Valley', 'Soup - Campbells Bean Medley', 'Wine - Blue Nun Qualitatswein', 'Chicken - Soup Base'], 200: ['Soup - Campbells Bean Medley', 'Muffin - Carrot Individual Wrap', 'Bay Leaf', 'Pork - Kidney', 'Wanton Wrap'], 264: ['Soupfoamcont12oz 112con', 'Wine - Two Oceans Cabernet', 'Bread - Italian Roll With Herbs', 'Veal - Inside, Choice', 'Fish - Scallops, Cold Smoked'], 356: ['Butter - Unsalted', 'Veal - Inside, Choice', 'Beets - Candy Cane, Organic', 'Nut - Chestnuts, Whole', 'Lamb - Ground'], 412: ['Olive - Spread Tapenade', 'Sprouts - Baby Pea Tendrils', 'Wine - Blue Nun Qualitatswein', 'Pepper - Black, Whole', 'Soup - Campbells Bean Medley']}


##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [95]:
step9 = pd.DataFrame.from_dict(products_recommended, orient= 'index')
step9

,0,1,2,3,4
33,Butter - Unsalted,Wine - Ej Gallo Sierra Valley,Soup - Campbells Bean Medley,Wine - Blue Nun Qualitatswein,Chicken - Soup Base
200,Soup - Campbells Bean Medley,Muffin - Carrot Individual Wrap,Bay Leaf,Pork - Kidney,Wanton Wrap
264,Soupfoamcont12oz 112con,Wine - Two Oceans Cabernet,Bread - Italian Roll With Herbs,"Veal - Inside, Choice","Fish - Scallops, Cold Smoked"
356,Butter - Unsalted,"Veal - Inside, Choice","Beets - Candy Cane, Organic","Nut - Chestnuts, Whole",Lamb - Ground
412,Olive - Spread Tapenade,Sprouts - Baby Pea Tendrils,Wine - Blue Nun Qualitatswein,"Pepper - Black, Whole",Soup - Campbells Bean Medley


In [99]:
one_hot_encoded_data = pd.get_dummies(step9, columns = [0, 1, 2, 3, 4])
one_hot_encoded_data

,0_Butter - Unsalted,0_Olive - Spread Tapenade,0_Soup - Campbells Bean Medley,0_Soupfoamcont12oz 112con,1_Muffin - Carrot Individual Wrap,1_Sprouts - Baby Pea Tendrils,"1_Veal - Inside, Choice",1_Wine - Ej Gallo Sierra Valley,1_Wine - Two Oceans Cabernet,2_Bay Leaf,...,"3_Nut - Chestnuts, Whole","3_Pepper - Black, Whole",3_Pork - Kidney,"3_Veal - Inside, Choice",3_Wine - Blue Nun Qualitatswein,4_Chicken - Soup Base,"4_Fish - Scallops, Cold Smoked",4_Lamb - Ground,4_Soup - Campbells Bean Medley,4_Wanton Wrap
33,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,1,0,0,0,0
200,0,0,1,0,1,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,1
264,0,0,0,1,0,0,0,0,1,0,...,0,0,0,1,0,0,1,0,0,0
356,1,0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,1,0,0
412,0,1,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [100]:
t_purchased_bis = data_purchased_pt.T

In [101]:
dist_calculation_bis = pdist(X = t_purchased , metric='correlation')
dist_calculation_bis

array([1.02521101, 0.8879431 , 0.95940007, ..., 0.98268152, 1.01783582,
       0.9766223 ])

In [102]:
dist_distribution_bis = squareform(dist_calculation_bis)
dist_distribution_bis

array([[0.        , 1.02521101, 0.8879431 , ..., 1.05568047, 1.03506774,
        1.03222313],
       [1.02521101, 0.        , 1.0010674 , ..., 0.99899044, 1.06495391,
        1.04609733],
       [0.8879431 , 1.0010674 , 0.        , ..., 0.98609845, 0.93897585,
        1.03717867],
       ...,
       [1.05568047, 0.99899044, 0.98609845, ..., 0.        , 0.98268152,
        1.01783582],
       [1.03506774, 1.06495391, 0.93897585, ..., 0.98268152, 0.        ,
        0.9766223 ],
       [1.03222313, 1.04609733, 1.03717867, ..., 1.01783582, 0.9766223 ,
        0.        ]])

In [103]:
euclid_dist_bis = pd.DataFrame(dist_distribution_bis,
                           index=data_purchased_pt.columns, 
                           columns=data_purchased_pt.columns)

euclid_dist_bis

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.000000,1.025211,0.887943,0.959400,0.972579,0.971542,1.051663,1.008305,1.003146,0.990052,...,1.026814,1.064510,1.026400,1.083910,1.000520,1.084359,1.027589,1.055680,1.035068,1.032223
200,1.025211,0.000000,1.001067,1.005715,1.073948,1.075028,0.926265,1.080738,1.003482,0.978399,...,0.981300,1.015439,0.930933,0.964814,1.030125,1.063924,0.870938,0.998990,1.064954,1.046097
264,0.887943,1.001067,0.000000,1.009730,0.947144,1.039546,0.987110,1.079281,0.991922,1.039807,...,0.965398,1.037471,1.001002,1.003357,0.931461,0.903754,0.953715,0.986098,0.938976,1.037179
356,0.959400,1.005715,1.009730,0.000000,1.040222,1.041743,0.992298,0.969268,1.071740,1.056114,...,1.003586,1.054821,1.005365,0.991127,1.023312,1.015781,1.065232,0.991223,0.974719,0.997480
412,0.972579,1.073948,0.947144,1.040222,0.000000,1.009738,0.979762,0.951334,1.024509,0.969367,...,1.048723,0.888568,1.032385,1.019126,0.981172,0.999745,1.077885,0.990375,0.976296,0.941690
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,1.084359,1.063924,0.903754,1.015781,0.999745,0.968645,0.981220,1.002805,0.998928,1.029966,...,1.021932,0.981406,1.050846,0.960573,0.870999,0.000000,0.902651,0.964935,1.032316,1.070734
98069,1.027589,0.870938,0.953715,1.065232,1.077885,1.065183,1.012509,1.039542,1.018858,0.976137,...,1.043515,1.001031,1.027032,1.063468,0.930410,0.902651,0.000000,0.954242,0.995756,1.004420
98159,1.055680,0.998990,0.986098,0.991223,0.990375,1.003224,0.999040,0.991565,0.959505,0.991348,...,0.979906,0.913048,0.997792,1.044248,0.974952,0.964935,0.954242,0.000000,0.982682,1.017836


In [104]:
euclid_dist_norm_bis = pd.DataFrame(1/(1 + squareform(pdist(t_purchased_bis, 'correlation'))),
                                index=t_purchased.index,
                                columns=t_purchased.index)

euclid_dist_norm_bis

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.493776,0.529677,0.510360,0.506951,0.507217,0.487410,0.497932,0.499215,0.502499,...,0.493385,0.484376,0.493486,0.479867,0.499870,0.479764,0.493197,0.486457,0.491384,0.492072
200,0.493776,1.000000,0.499733,0.498575,0.482172,0.481921,0.519139,0.480599,0.499131,0.505459,...,0.504719,0.496170,0.517884,0.508954,0.492581,0.484514,0.534491,0.500253,0.484272,0.488735
264,0.529677,0.499733,1.000000,0.497579,0.513573,0.490305,0.503243,0.480936,0.502028,0.490242,...,0.508803,0.490805,0.499750,0.499162,0.517743,0.525278,0.511845,0.503500,0.515736,0.490875
356,0.510360,0.498575,0.497579,1.000000,0.490143,0.489778,0.501933,0.507803,0.482686,0.486354,...,0.499105,0.486660,0.498662,0.502228,0.494239,0.496086,0.484207,0.502204,0.506401,0.500631
412,0.506951,0.482172,0.513573,0.490143,1.000000,0.497577,0.505111,0.512470,0.493947,0.507777,...,0.488109,0.529502,0.492033,0.495264,0.504752,0.500064,0.481259,0.502418,0.505997,0.515015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.479764,0.484514,0.525278,0.496086,0.500064,0.507963,0.504739,0.499300,0.500268,0.492619,...,0.494577,0.504692,0.487604,0.510055,0.534474,1.000000,0.525583,0.508923,0.492050,0.482921
98069,0.493197,0.534491,0.511845,0.484207,0.481259,0.484219,0.496892,0.490306,0.495330,0.506038,...,0.489353,0.499742,0.493332,0.484621,0.518025,0.525583,1.000000,0.511707,0.501063,0.498897
98159,0.486457,0.500253,0.503500,0.502204,0.502418,0.499195,0.500240,0.502118,0.510333,0.502173,...,0.505074,0.522726,0.500553,0.489177,0.506341,0.508923,0.511707,1.000000,0.504367,0.495580


In [105]:
products_recommended = {}

for i in range(5):
    euclid_dist_norm_2 = euclid_dist_norm_bis.iloc[:,i]
    top_cust = euclid_dist_norm_2.to_frame()
    column_name = top_cust.columns
    top_cust = top_cust.sort_values(column_name[0], ascending = False)
    top_5cust = top_cust.iloc[0:6]
    top_5cust.reset_index(inplace=True)
    top_5cust = top_5cust[top_5cust['CustomerID']!= column_name[0]]
    top_5cust = top_5cust.iloc[:,0]
    top_5cust = list(top_5cust)
    step6 = data_purchased[np.in1d(data_purchased.index.get_level_values(0), top_5cust)]
    step6 = step6.reset_index()
    step6 = step6[['ProductName', 'Quantity']]
    step6_sum = step6.groupby('ProductName').sum().sort_values('Quantity', ascending = False)
    step7 = data_purchased.reset_index()
    step7 = step7[step7['CustomerID'] == column_name[0]]
    step7_result = step6_sum.merge(step7, on='ProductName', how='left')
    step7_result = step7_result[step7_result['CustomerID']!= column_name[0]]
    products = step7_result.iloc[:5]
    products_recommended[column_name[0]] = list(products['ProductName'])
    

print(products_recommended)

{33: ['Knife Plastic - White', 'Muffin - Zero Transfat', 'Banana Turning', 'Crush - Cream Soda', 'Veal - Osso Bucco'], 200: ['Otomegusa Dashi Konbu', 'Milk Powder', 'Potatoes - Idaho 100 Count', 'Crackers - Trio', 'Pail With Metal Handle 16l White'], 264: ['Water - Mineral, Natural', 'Wine - Toasted Head', 'Snapple - Iced Tea Peach', 'Pickerel - Fillets', 'Garbag Bags - Black'], 356: ['Cheese - Taleggio D.o.p.', 'Coconut - Shredded, Sweet', 'Cheese - Cheddarsliced', 'Ocean Spray - Kiwi Strawberry', 'Olives - Kalamata'], 412: ['Cake - Mini Cheesecake', 'Butter - Unsalted', 'Salmon - Atlantic, Skin On', 'Wine - Hardys Bankside Shiraz', 'Gloves - Goldtouch Disposable']}


In [106]:
step11 = pd.DataFrame.from_dict(products_recommended, orient= 'index')
step11

,0,1,2,3,4
33,Knife Plastic - White,Muffin - Zero Transfat,Banana Turning,Crush - Cream Soda,Veal - Osso Bucco
200,Otomegusa Dashi Konbu,Milk Powder,Potatoes - Idaho 100 Count,Crackers - Trio,Pail With Metal Handle 16l White
264,"Water - Mineral, Natural",Wine - Toasted Head,Snapple - Iced Tea Peach,Pickerel - Fillets,Garbag Bags - Black
356,Cheese - Taleggio D.o.p.,"Coconut - Shredded, Sweet",Cheese - Cheddarsliced,Ocean Spray - Kiwi Strawberry,Olives - Kalamata
412,Cake - Mini Cheesecake,Butter - Unsalted,"Salmon - Atlantic, Skin On",Wine - Hardys Bankside Shiraz,Gloves - Goldtouch Disposable
